## Study Compas Dataset  
[Link](https://github.com/propublica/compas-analysis/blob/master/Compas%20Analysis.ipynb)

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
DATASET_PATH = os.path.join("dataset", "compas", "compas-scores.csv")

In [3]:
data = pd.read_csv(DATASET_PATH)
data.head(5)

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,vr_offense_date,vr_charge_desc,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,type_of_assessment,decile_score.1,score_text,screening_date
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,NaN,NaN,Risk of Violence,1,Low,2013-08-14,Risk of Recidivism,1,Low,2013-08-14
1,2,michael ryan,michael,ryan,2014-12-31,Male,1985-02-06,31,25 - 45,Caucasian,...,NaN,NaN,Risk of Violence,2,Low,2014-12-31,Risk of Recidivism,5,Medium,2014-12-31
2,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,2013-07-05,Felony Battery (Dom Strang),Risk of Violence,1,Low,2013-01-27,Risk of Recidivism,3,Low,2013-01-27
3,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,NaN,NaN,Risk of Violence,3,Low,2013-04-14,Risk of Recidivism,4,Low,2013-04-14
4,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,NaN,NaN,Risk of Violence,6,Medium,2013-01-13,Risk of Recidivism,8,High,2013-01-13


In [4]:
data.shape

(11757, 47)

In [5]:
data.columns

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'num_r_cases',
       'r_case_number', 'r_charge_degree', 'r_days_from_arrest',
       'r_offense_date', 'r_charge_desc', 'r_jail_in', 'r_jail_out',
       'is_violent_recid', 'num_vr_cases', 'vr_case_number',
       'vr_charge_degree', 'vr_offense_date', 'vr_charge_desc',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'type_of_assessment', 'decile_score.1',
       'score_text', 'screening_date'],
      dtype='object')

In [6]:
def check_data(df, columns):
    for col in columns:
        print(col + ":")
        if not col in df.columns:
            print("Nothing found")
        else:
            print(df[col].unique())
        print()

In [7]:
check_data(data, ['age', 'c_charge_degree', 'race', 'age_cat', 'score_text', 'sex', 'priors_count', 
                    'days_b_screening_arrest', 'decile_score', 'is_recid', 'two_year_recid', 'c_jail_in', 'c_jail_out']
          )

age:
[69 31 34 24 23 43 44 41 39 20 26 21 27 37 22 47 25 64 42 32 54 49 36 33
 30 63 55 29 38 53 51 28 35 48 46 62 56 45 40 59 50 66 52 71 19 61 58 78
 57 68 18 70 60 65 83 67 75 73 72 77 76 74 96 80 79 86]

c_charge_degree:
['F' 'O' 'M']

race:
['Other' 'Caucasian' 'African-American' 'Hispanic' 'Asian'
 'Native American']

age_cat:
['Greater than 45' '25 - 45' 'Less than 25']

score_text:
['Low' 'Medium' 'High' nan]

sex:
['Male' 'Female']

priors_count:
[ 0  4  1  2 14  3  7  6  5 13  8  9 15 19 21 22 20 10 12 28 11 23 25 24
 36 18 29 16 17 33 30 43 27 26 38 37 35 31 39]

days_b_screening_arrest:
[-1.000e+00        nan  0.000e+00  4.280e+02 -2.000e+01  2.200e+01
 -2.000e+00 -1.320e+02  5.300e+01  8.100e+01 -5.900e+01 -2.400e+01
  3.020e+02 -2.190e+02 -1.300e+01 -3.800e+01  7.000e+01 -3.250e+02
 -1.500e+01 -7.800e+01 -4.300e+01 -3.890e+02 -3.180e+02 -5.000e+00
  1.370e+02 -1.810e+02 -2.390e+02 -2.100e+01 -6.400e+01 -1.000e+01
 -1.370e+02  1.780e+02 -3.000e+01 -3.000e+00  6.700e+01  1.

In [8]:
# select data and preprocess
data_processed = data[data['days_b_screening_arrest'] <= 30]
data_processed = data[data['days_b_screening_arrest'] >= -30] # select within 30 days
data_processed.shape

(9778, 47)

In [9]:
data_processed = data_processed[data_processed['is_recid'] != -1]
data_processed = data_processed[data_processed['c_charge_degree'] != 'O']
data_processed.dropna(subset=["score_text"], inplace=True)
data_processed.shape

(9760, 47)

In [10]:
data_train = data_processed[["age_cat", "c_charge_degree", "race", "sex", "priors_count"]].copy()
data_compare = data_processed[["score_text", "decile_score"]].copy()
data_target = data_processed[["is_recid"]].copy()

In [11]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9760 entries, 0 to 11756
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age_cat          9760 non-null   object
 1   c_charge_degree  9760 non-null   object
 2   race             9760 non-null   object
 3   sex              9760 non-null   object
 4   priors_count     9760 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 457.5+ KB


In [12]:
data_compare.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9760 entries, 0 to 11756
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   score_text    9760 non-null   object
 1   decile_score  9760 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 228.8+ KB


In [13]:
data_target.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9760 entries, 0 to 11756
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   is_recid  9760 non-null   int64
dtypes: int64(1)
memory usage: 152.5 KB


In [14]:
data_target = data_target.to_numpy().ravel()

In [15]:
# preprocessing
from sklearn.preprocessing import LabelEncoder
encoders = {}
for col in ['age_cat', 'c_charge_degree', 'race', 'sex']:
    encoder = LabelEncoder()
    data_train[col] = encoder.fit_transform(data_train[col])
    encoders[col] = encoder
data_train.head(5)

,age_cat,c_charge_degree,race,sex,priors_count
0,1,0,5,1,0
2,0,0,0,1,0
3,2,0,0,1,4
6,0,1,5,1,0
7,0,0,2,1,14


In [16]:
data_train.describe()

,age_cat,c_charge_degree,race,sex,priors_count
count,9760.000000,9760.000000,9760.000000,9760.000000,9760.000000
mean,0.642725,0.347848,1.267623,0.795389,3.128586
std,0.811283,0.476312,1.450518,0.403437,4.670708
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,0.000000
50%,0.000000,0.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,2.000000,1.000000,4.000000
max,2.000000,1.000000,5.000000,1.000000,43.000000


In [17]:
data_train = data_train.to_numpy()

## Build Model  
The goal here is to build several models to predict the possibility of a person's is_recid in future  
and compare it with the decile_score and score_text output  

In [18]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

In [19]:
result = cross_validate(estimator=KNeighborsClassifier(),
                        X=data_train, y=data_target, cv=10,
                        return_estimator=True)
result_best_ids = np.argmax(result["test_score"])
model_neighbor = {
    'avg_score': result['test_score'].mean(),
    'best_estimator': result['estimator'][result_best_ids]
}
print(model_neighbor['avg_score'])

0.6420081967213115


In [20]:
result = cross_validate(estimator=LinearSVC(dual=False),
                        X=data_train, y=data_target, cv=10,
                        return_estimator=True)
result_best_ids = np.argmax(result["test_score"])
model_linearSVC = {
    'avg_score': result['test_score'].mean(),
    'best_estimator': result['estimator'][result_best_ids]
}
print(model_linearSVC['avg_score'])

0.6762295081967213


In [21]:
result = cross_validate(estimator=SGDClassifier(),
                        X=data_train, y=data_target, cv=10,
                        return_estimator=True)
result_best_ids = np.argmax(result["test_score"])
model_SGD = {
    'avg_score': result['test_score'].mean(),
    'best_estimator': result['estimator'][result_best_ids]
}
print(model_SGD['avg_score'])

0.6682377049180329


In [22]:
result = cross_validate(estimator=LogisticRegression(),
                        X=data_train, y=data_target, cv=10,
                        return_estimator=True)
result_best_ids = np.argmax(result["test_score"])
model_logreg = {
    'avg_score': result['test_score'].mean(),
    'best_estimator': result['estimator'][result_best_ids]
}
print(model_logreg['avg_score'])

0.6774590163934426


In [23]:
result = cross_validate(estimator=RandomForestClassifier(),
                        X=data_train, y=data_target, cv=10,
                        return_estimator=True)
result_best_ids = np.argmax(result["test_score"])
model_forest = {
    'avg_score': result['test_score'].mean(),
    'best_estimator': result['estimator'][result_best_ids]
}
print(model_forest['avg_score'])

0.6701844262295082


### Comparison

In [24]:
sample_index = 25
print(data_train[sample_index])
print(model_neighbor["best_estimator"].predict([data_train[sample_index]]))
print(model_neighbor["best_estimator"].predict_proba([data_train[sample_index]]))

[2 0 3 1 1]
[1]
[[0.2 0.8]]
